# Migration simulation in EU - results analysis

Required properties (**must be the same as in migration_simulation.ipynb!**).

In [1]:
countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark',
 'Estonia', 'Finland', 'France', 'Greece', 'Spain', 'Ireland', 'Lithuania', 'Luxemburg', 'Latvia',
 'Malta', 'Netherlands', 'Germany', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Sweden',
 'Hungary', 'United Kingdom', 'Italy']

In [2]:
n_iter = 50

## Data loading

Load data required for statistics.

In [3]:
import pandas as pd

def read_data_from_csv(filename: str) -> pd.DataFrame:
    return pd.read_csv('results/' + filename + '.csv', sep=';')


agent_current_countries = read_data_from_csv('current_country')
agent_races = read_data_from_csv('race')
agent_religious_denomination = read_data_from_csv('religious_denomination')

## Analysis

### Country

Below we can see current country for agents in each step.

In [4]:
df_result_visited_countries = pd.DataFrame.from_records(
    [agent['current_country'].tolist() for _, agent in agent_current_countries.reset_index().groupby(['AgentID'])])
display(df_result_visited_countries.head())

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,Slovakia,Hungary,Slovakia,Czech Republic,Austria,Slovakia,Poland,Slovakia,Czech Republic,Poland,...,France,Germany,Czech Republic,Czech Republic,Austria,Slovakia,Slovakia,Slovakia,Poland,Sweden
1,Poland,Sweden,Sweden,Latvia,Sweden,Sweden,Denmark,Sweden,Denmark,Germany,...,Cyprus,Cyprus,Cyprus,Greece,Cyprus,Greece,Bulgaria,Romania,Romania,Romania
2,United Kingdom,Germany,Germany,Denmark,Denmark,Denmark,Germany,France,Luxemburg,Belgium,...,United Kingdom,United Kingdom,United Kingdom,France,Germany,Netherlands,Netherlands,Ireland,Belgium,Germany
3,Sweden,Latvia,Estonia,Latvia,Latvia,Sweden,Poland,Poland,Lithuania,Lithuania,...,Netherlands,Ireland,Belgium,Ireland,Ireland,Netherlands,Ireland,Netherlands,Ireland,Ireland
4,Czech Republic,Austria,Austria,Austria,Germany,Sweden,Estonia,Latvia,Lithuania,Lithuania,...,Germany,United Kingdom,Netherlands,France,Belgium,Luxemburg,Belgium,Netherlands,Netherlands,Belgium


#### The same first and last country
Based on result, more agents feel happier in different country than their origin one. <br>

In [5]:
(df_result_visited_countries.iloc[:, 0] == df_result_visited_countries.iloc[:, n_iter - 1]).value_counts()

False    95027
True      4973
dtype: int64

#### Number of visited countries

In [6]:
df_result_visited_countries['unique_countries_count'] = df_result_visited_countries.apply(pd.Series.nunique, axis=1)

Max number of visited country for agent is:

In [7]:
df_result_visited_countries['unique_countries_count'].max()

25

It is half of possible movements. <br>
Agents can go ahead (e.g. to check whether life is better in another country) and go back.

Min number of visited country for agent is:

In [8]:
df_result_visited_countries['unique_countries_count'].min()

3

There is no agent who didn't change country within 50 iterations.
It means that agents were willingly to discover new places.
They tried something new but could realize that previous place was better.

Median number of visited country for agent is:

In [9]:
df_result_visited_countries['unique_countries_count'].median()

15.0

Median is lower than half of number of possible movements. It can indicate that agent found the best place and didn't want to try another one.

### Race

In simulation I had following number of agents with race:

In [10]:
df_result_race = pd.DataFrame.from_records(
    [agent['race'].tolist() for _, agent in agent_races.reset_index().groupby(['AgentID'])])
df_result_race.iloc[:, 0].value_counts()

White    95827
Black     2269
Asian     1904
Name: 0, dtype: int64

About 95.8% of agents were white people. <br>
About 2.3% of agents were black people. <br>
And the rest of agents (1.9%) were asian people.

There were a similar number of asian and black people which sum is much smaller than number of white people.

### Religious denomination

In simulation I had following number of agents with religious denomination:

In [11]:
df_result_religious_denomination = pd.DataFrame.from_records(
    [agent['religious_denomination'].tolist() for _, agent in
     agent_religious_denomination.reset_index().groupby(['AgentID'])])
df_result_religious_denomination.iloc[:, 0].value_counts()

Christian    92633
Muslim        7073
Jew            294
Name: 0, dtype: int64

Christians (92.6%) are the most numerous religious in simulation. Then are Muslims (7.1%) and Jews (0.3%).

### Correlations

Following section provides an information about correlation between data.

In [12]:
df_gathered_results = pd.DataFrame({'origin_country': df_result_visited_countries.iloc[:, 0],
                                    'last_country': df_result_visited_countries.iloc[:, n_iter - 1],
                                    'race': df_result_race.iloc[:, 0],
                                    'religious_denomination': df_result_religious_denomination.iloc[:, 0]
                                    })

#### Race - the same origin and last country

- White people

In [13]:
df_gathered_results_white_people = df_gathered_results.groupby('race').get_group('White') 
(df_gathered_results_white_people['origin_country'] == df_gathered_results_white_people['last_country']).value_counts()

False    91041
True      4786
dtype: int64

White people were totally likely to travel, more than 95% of them have different origin and last country.

- Black people

In [14]:
df_gathered_results_black_people = df_gathered_results.groupby('race').get_group('Black') 
(df_gathered_results_black_people['origin_country'] == df_gathered_results_black_people['last_country']).value_counts()

False    2173
True       96
dtype: int64

About 95.8% of black people have different origin and last country in simulation.

- Asian people

In [15]:
df_gathered_results_asian_people = df_gathered_results.groupby('race').get_group('Asian') 
(df_gathered_results_asian_people['origin_country'] == df_gathered_results_asian_people['last_country']).value_counts()

False    1813
True       91
dtype: int64

About 95.3% of jews travel around the EU.

As we can see, simillar number of agents representing races have migrated to another country.

#### Religious denomination - the same origin and last country

- Christians

In [16]:
df_gathered_results_christians = df_gathered_results.groupby('religious_denomination').get_group('Christian') 
(df_gathered_results_christians['origin_country'] == df_gathered_results_christians['last_country']).value_counts()

False    88117
True      4516
dtype: int64

The vast majority of Christians (95.1%) changed the country.

- Muslims

In [17]:
df_gathered_results_muslims = df_gathered_results.groupby('religious_denomination').get_group('Muslim') 
(df_gathered_results_muslims['origin_country'] == df_gathered_results_muslims['last_country']).value_counts()

False    6640
True      433
dtype: int64

93.9% of Muslims considered another country as a new place for living.

- Jews

In [18]:
df_gathered_results_jews = df_gathered_results.groupby('religious_denomination').get_group('Jew') 
(df_gathered_results_jews['origin_country'] == df_gathered_results_jews['last_country']).value_counts()

False    270
True      24
dtype: int64

91.74% of Jews found to be more happy in another country.

As we can see, traveling is not linked with any kind of religious denomination.